In [1]:
import os
os.chdir("..")
os.listdir()

['pyproject.toml',
 'README.md',
 '.gitignore',
 '.venv',
 'poetry.lock',
 '.git',
 'data',
 'notebooks',
 'reports',
 'src']

In [2]:
import gurobipy
from gurobipy import Model, GRB, quicksum


def build_model_multi_hotspot(
    H, # Set of hotspots
    S, # Set of shelters
    G, # Set of genders
    d_hg, # {(hotspot, gender): demand at hotspot h for gender g}
    c, # {shelter: capacity}
    dist, # {(hotspot, shelter): distance}
    e, # {(shelter, gender): 1 if eligible, 0 else}
):
    m = Model("Optimization_Multi_Hotspot")

    # Decision vars: x_hsg and z_hg
    x = {(h, s, g): m.addVar(vtype=GRB.INTEGER , lb=0.0, name=f"x_{h}_{s}_{g}") for h in H for s in S for g in G}
    
    z = {(h, g): m.addVar(vtype=GRB.INTEGER , lb=0.0, name=f"z_{h}_{g}") for h in H for g in G}

    # Good practice
    m.update()

    '''
    Demand balance for each hotspot. For every hotspot, the # of people sent to all shelters 
    plus the number of people left behind must equal the total demand for each hotspot
    '''
    # Now looping through G as well to balance demand for each gender
    for h in H:
        for g in G:
            m.addConstr(quicksum(x[(h, s, g)] for s in S) + z[(h, g)] == d_hg[(h, g)], name=f"demand_{h}_{g}")

    '''
    Capacity for each shelter. A shelter cannot accept more people than its capacity. Sum of people coming from
    all the hotspots to shelter s must be less than capacity c_s
    '''
    # We sum up all people (h) of all genders (g) arriving at shelter (s)
    for s in S:
        m.addConstr(quicksum(x[(h, s, g)] for h in H for g in G) <= c[s], name=f"cap_{s}")

    '''
    Eligibility constraints. If a shelter cannot accept a specific gender (e=0), 
    force the flow (x) to that shelter for that gender to be 0.
    '''
    for s in S:
        for g in G:
            if e[(s, g)] == 0:
                for h in H:
                    m.addConstr(x[(h, s, g)] == 0, name=f"elig_{s}_{g}_{h}")
# ---------------------------------------------------------
    # Step 1: Min total unsheltered
    # ---------------------------------------------------------
    m.setObjective(quicksum(z[(h, g)] for h in H for g in G), GRB.MINIMIZE)
    m.optimize()

    # Capture the Z value PER GENDER
    Z_star_per_gender = {}
    for g in G:
        Z_star_per_gender[g] = sum(z[(h, g)].X for h in H)

    # Total Z (just for reporting)
    Z_star_total = sum(Z_star_per_gender.values())

    # ---------------------------------------------------------
    # Step 2: Fix unsheltered PER GENDER and minimize distance
    # ---------------------------------------------------------
    
    # We add a constraint for EACH gender separately
    # "Men unserved must equal Stage 1 Men unserved"
    # "Women unserved must equal Stage 1 Women unserved"
    for g in G:
        m.addConstr(
            quicksum(z[(h, g)] for h in H) == Z_star_per_gender[g], 
            name=f"fix_unsheltered_{g}"
        )

    # Now minimize distance
    m.setObjective(quicksum(dist[(h, s)] * x[(h, s, g)] for h in H for s in S for g in G), GRB.MINIMIZE)
    m.optimize()

    return m, {"x": x, "z": z}, {"Z_star": Z_star_total, "obj_distance": m.ObjVal}

In [3]:
def run_referral_simulation(
    monthly_demands, hotspots, shelters, genders, 
    capacities, eligibility, distances, print_utilization=True
):
    results = []

    for month_idx, demand_data in enumerate(monthly_demands, start=1):
        
        # --- 1. PREPARE DATA ---
        demand_flat = {}
        total_monthly_demand = 0.0
        
        for h in hotspots:
            for g in genders:
                val = demand_data[h][g]
                demand_flat[(h, g)] = val
                total_monthly_demand += val

        # --- 2. RUN OPTIMIZATION ---
        model, variables, info = build_model_multi_hotspot(
            hotspots, shelters, genders, demand_flat, 
            capacities, distances, eligibility
        )
        
        if not variables: 
            print(f"Month {month_idx} Infeasible.")
            continue

        x_vars = variables["x"]
        z_vars = variables["z"]

        # --- 3. EXTRACT RESULTS ---
        total_assigned_count = 0.0
        for h in hotspots:
            for s in shelters:
                for g in genders:
                    total_assigned_count += x_vars[(h, s, g)].X

        # Overall Avg Distance
        avg_dist_month = 0.0
        if total_assigned_count > 0:
            avg_dist_month = info["obj_distance"] / total_assigned_count

        # --- 4. CALCULATE GENDER SPECIFIC STATS ---
        gender_stats = {}
        for g in genders:
            # 1. Count Unsheltered
            g_unsheltered = sum(z_vars[(h, g)].X for h in hotspots)
            
            # 2. Count Sheltered (Assigned)
            g_assigned = sum(x_vars[(h, s, g)].X for h in hotspots for s in shelters)
            
            # 3. Calculate Distance for this gender only
            g_total_dist = 0.0
            for h in hotspots:
                for s in shelters:
                    flow = x_vars[(h, s, g)].X
                    if flow > 0.001: # Optimization check for non-zero flow
                        g_total_dist += flow * distances[(h, s)]
            
            g_avg_dist = (g_total_dist / g_assigned) if g_assigned > 0 else 0.0

            gender_stats[g] = {
                "assigned": g_assigned,
                "unsheltered": g_unsheltered,
                "avg_dist": g_avg_dist
            }

        # --- 5. SHELTER UTILIZATION ---
        shelter_usage_stats = {}
        for s in shelters:
            used_amount = sum(x_vars[(h, s, g)].X for h in hotspots for g in genders)
            total_cap = capacities[s]
            pct_full = (used_amount / total_cap) * 100.0 if total_cap > 0 else 0.0
            
            shelter_usage_stats[s] = {
                "used": used_amount,
                "capacity": total_cap,
                "utilization_pct": pct_full,
            }

        # --- 6. PRINTING (Monthly Log) ---
        if print_utilization:
            print(f"\n{'='*40}")
            print(f"MONTH {month_idx}")
            print(f"{'='*40}")
            print(f"Total Demand: {total_monthly_demand:.0f}")
            print(f"Total Unserved: {info['Z_star']:.0f}")
            
            for g in genders:
                stats = gender_stats[g]
                print(f"  > {g.capitalize()}: Assigned {stats['assigned']:.0f} | Unserved {stats['unsheltered']:.0f} | Avg Dist {stats['avg_dist']:.3f} km")

        # Store Results
        month_result = {
            "month": month_idx,
            "total_unsheltered": info["Z_star"],
            "total_distance": info["obj_distance"],
            "total_assigned": total_assigned_count,
            "overall_avg_distance": avg_dist_month,
            "shelter_usage": shelter_usage_stats,
            "gender_breakdown": gender_stats # Contains the specific stats we need
        }
        results.append(month_result)

    return results

In [4]:
if __name__ == "__main__":
    import pandas as pd
    import sys
    # Assuming src.config is available in your environment
    # If running locally without the src module, replace PROCESSED_DATA_DIR with a string path like "."
    try:
        from src.config import PROCESSED_DATA_DIR
    except ImportError:
        from pathlib import Path
        PROCESSED_DATA_DIR = Path(".") # Fallback to current directory

    # ==========================================
    # 1. LOAD DATA FRAMES
    # ==========================================
    print("Loading Data...")
    df_demand = pd.read_csv(PROCESSED_DATA_DIR / 'demand_per_hotspot.csv')
    df_dist = pd.read_csv(PROCESSED_DATA_DIR / 'distance_amtrix.csv')
    df_capacity = pd.read_csv(PROCESSED_DATA_DIR / 'capacities_per_shelter.csv')
    df_genders = pd.read_csv(PROCESSED_DATA_DIR / "eligibility_df.csv")

    # ==========================================
    # 2. PROCESS DATAFRAMES INTO MODEL INPUTS
    # ==========================================
    
    print("Processing Data...")
    genders = ["men", "women"]
    
    # --- A. Define Hotspots ---
    # Hotspots are the columns in the distance matrix (excluding the name column)
    # Filter out any non-hotspot columns like 'Unnamed: 0' if they exist
    hotspots = [col for col in df_dist.columns if col != 'LOCATION_NAME' and not col.startswith('Unnamed')]

    # --- B. Process Shelters, Capacities & Eligibility ---
    shelters = []
    capacities = {}
    eligibility = {}
    
    # Map Location Name -> List of Shelter IDs 
    # (Critical because "Good Shepherd" appears multiple times with different IDs/Caps but one distance row)
    location_name_to_ids = {}

    # Iterate through capacities. We assume df_capacity and df_genders 
    # are sorted identically (row 0 in cap corresponds to row 0 in gender).
    for idx, row_cap in df_capacity.iterrows():
        # Use the DataFrame Index as the unique Shelter ID
        s_id = idx
        
        name = row_cap['SHELTER_NAME']
        cap = float(row_cap['CAPACITY'])
        
        # Get corresponding eligibility row by index
        row_elig = df_genders.iloc[idx] 
        
        # Verification: Ensure rows align by name
        if row_elig['SHELTER_NAME'] != name:
            # If they don't align, we try to find the matching row (safer)
            matching_rows = df_genders[df_genders['SHELTER_NAME'] == name]
            if not matching_rows.empty:
                row_elig = matching_rows.iloc[0]
            else:
                print(f"Warning: Could not find gender info for {name}")
                continue

        shelters.append(s_id)
        capacities[s_id] = cap
        
        # Extract Eligibility (1 or 0)
        eligibility[(s_id, "men")] = int(row_elig['men_eligible_eligible'])
        eligibility[(s_id, "women")] = int(row_elig['women_eligible_eligible'])
        
        # Build Map: Name -> [List of IDs]
        if name not in location_name_to_ids:
            location_name_to_ids[name] = []
        location_name_to_ids[name].append(s_id)

    # --- C. Process Distances ---
    distances = {}
    
    for idx, row in df_dist.iterrows():
        loc_name = row['LOCATION_NAME']
        
        # Check if this distance location exists in our shelter data
        if loc_name in location_name_to_ids:
            associated_ids = location_name_to_ids[loc_name]
            
            # Loop through all hotspots found in columns
            for h in hotspots:
                if h in df_dist.columns:
                    # Get distance value
                    dist_val = float(row[h])
                    
                    # Assign this distance to ALL unique shelter IDs that share this Location Name
                    for s_id in associated_ids:
                        distances[(h, s_id)] = dist_val

    # Failsafe: Fill missing distances with a high penalty
    # This happens if a shelter in the capacity file doesn't have a matching name in the distance file
    for h in hotspots:
        for s_id in shelters:
            if (h, s_id) not in distances:
                # print(f"Warning: Missing distance for {s_id} - {h}")
                distances[(h, s_id)] = 999.9 

    # --- D. Process Monthly Demand ---
    monthly_demands = []
    
    # Get sorted months (1 to 12)
    sorted_months = sorted(df_demand['month'].unique())
    
    for m in sorted_months:
        month_dict = {}
        # Filter dataframe for specific month
        df_m = df_demand[df_demand['month'] == m]
        
        for idx, row in df_m.iterrows():
            h = row['CFSAUID']
            
            # Only process if this hotspot exists in our defined hotspots list
            if h in hotspots:
                month_dict[h] = {
                    "men": float(row['male_from_hotspot']),
                    "women": float(row['female_from_hotspot'])
                }
        monthly_demands.append(month_dict)

    # ==========================================
    # 3. RUN SIMULATION
    # ==========================================
    print("\nStarting Simulation...")
    results = run_referral_simulation(
        monthly_demands=monthly_demands,
        hotspots=hotspots,
        shelters=shelters,
        genders=genders,
        capacities=capacities,
        eligibility=eligibility,
        distances=distances,
        print_utilization=True
    )

# ==========================================
    # 4. FINAL SUMMARY
    # ==========================================
    print("\n" + "="*100)
    print(f"{'YEARLY SUMMARY':^100}")
    print("="*100)
    
    # Header Row
    print(f"{'Month':<6} | {'TOTAL':<15} || {'MEN':<23} || {'WOMEN':<23}")
    print(f"{'':<6} | {'Unshelt':<7} {'AvgKM':<7} || {'Shelt':<6} {'Unshelt':<8} {'AvgKM':<7} || {'Shelt':<6} {'Unshelt':<8} {'AvgKM':<7}")
    print("-" * 100)

    for r in results:
        # Extract Total Stats
        tot_un = r['total_unsheltered']
        tot_dist = r['overall_avg_distance']
        
        # Extract Men Stats
        m_stats = r['gender_breakdown']['men']
        m_shelt = m_stats['assigned']
        m_un = m_stats['unsheltered']
        m_dist = m_stats['avg_dist']
        
        # Extract Women Stats
        w_stats = r['gender_breakdown']['women']
        w_shelt = w_stats['assigned']
        w_un = w_stats['unsheltered']
        w_dist = w_stats['avg_dist']

        print(
            f"{r['month']:<6} | "
            f"{tot_un:<7.0f} {tot_dist:<7.3f} || "
            f"{m_shelt:<6.0f} {m_un:<8.0f} {m_dist:<7.3f} || "
            f"{w_shelt:<6.0f} {w_un:<8.0f} {w_dist:<7.3f}"
        )

Loading Data...
Processing Data...

Starting Simulation...
Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 967 rows, 1792 columns and 4432 nonzeros (Min)
Model fingerprint: 0xe9ead36d
Model has 32 linear objective coefficients
Variable types: 0 continuous, 1792 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 8050.0000000
Presolve removed 916 rows and 1216 columns
Presolve time: 0.01s
Presolved: 51 rows, 576 columns, 1136 nonzeros
Found heuristic solution: objective 7292.0000000
Variable types: 0 continuous, 576 integer (32 binary)

Root relaxation: objective 5.535000e+03, 98 iterations, 0.00 seconds

#### Correct Sim Random Data

In [5]:
results = [
    {
        "month": 1,
        "total_unsheltered": 5535,
        "overall_avg_distance": 9.722,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3716, "avg_dist": 11.062},
            "women": {"assigned": 758, "unsheltered": 1819, "avg_dist": 6.617}
        }
    },
    {
        "month": 2,
        "total_unsheltered": 5571,
        "overall_avg_distance": 7.968,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3729, "avg_dist": 6.303},
            "women": {"assigned": 758, "unsheltered": 1842, "avg_dist": 11.827}
        }
    },
    {
        "month": 3,
        "total_unsheltered": 5065,
        "overall_avg_distance": 7.220,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3388, "avg_dist": 6.957},
            "women": {"assigned": 758, "unsheltered": 1677, "avg_dist": 7.830}
        }
    },
    {
        "month": 4,
        "total_unsheltered": 5079,
        "overall_avg_distance": 7.545,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3371, "avg_dist": 7.264},
            "women": {"assigned": 758, "unsheltered": 1708, "avg_dist": 8.197}
        }
    },
    {
        "month": 5,
        "total_unsheltered": 4680,
        "overall_avg_distance": 9.331,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3066, "avg_dist": 9.922},
            "women": {"assigned": 758, "unsheltered": 1614, "avg_dist": 7.960}
        }
    },
    {
        "month": 6,
        "total_unsheltered": 4526,
        "overall_avg_distance": 7.645,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2939, "avg_dist": 6.689},
            "women": {"assigned": 758, "unsheltered": 1587, "avg_dist": 9.861}
        }
    },
    {
        "month": 7,
        "total_unsheltered": 4455,
        "overall_avg_distance": 7.633,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2852, "avg_dist": 7.406},
            "women": {"assigned": 758, "unsheltered": 1603, "avg_dist": 8.159}
        }
    },
    {
        "month": 8,
        "total_unsheltered": 4430,
        "overall_avg_distance": 9.650,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2836, "avg_dist": 10.512},
            "women": {"assigned": 758, "unsheltered": 1594, "avg_dist": 7.653}
        }
    },
    {
        "month": 9,
        "total_unsheltered": 4260,
        "overall_avg_distance": 9.105,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2732, "avg_dist": 8.915},
            "women": {"assigned": 758, "unsheltered": 1528, "avg_dist": 9.544}
        }
    },
    {
        "month": 10,
        "total_unsheltered": 4251,
        "overall_avg_distance": 9.316,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2705, "avg_dist": 9.080},
            "women": {"assigned": 758, "unsheltered": 1546, "avg_dist": 9.864}
        }
    },
    {
        "month": 11,
        "total_unsheltered": 4540,
        "overall_avg_distance": 8.341,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2934, "avg_dist": 7.603},
            "women": {"assigned": 758, "unsheltered": 1606, "avg_dist": 10.050}
        }
    },
    {
        "month": 12,
        "total_unsheltered": 5514,
        "overall_avg_distance": 8.288,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3690, "avg_dist": 8.633},
            "women": {"assigned": 758, "unsheltered": 1824, "avg_dist": 7.488}
        }
    }
]

# ==========================================
# 4. FINAL SUMMARY
# ==========================================
print("\n" + "="*100)
print(f"{'YEARLY SUMMARY':^100}")
print("="*100)

# Header Row - Updated widths for MEN and WOMEN to 23 to match sub-columns
print(f"{'Month':<6} | {'TOTAL':<15} || {'MEN':<23} || {'WOMEN':<23}")
print(f"{'':<6} | {'Unshelt':<7} {'AvgKM':<7} || {'Shelt':<6} {'Unshelt':<8} {'AvgKM':<7} || {'Shelt':<6} {'Unshelt':<8} {'AvgKM':<7}")
print("-" * 100)

for r in results:
    # Extract Total Stats
    tot_un = r['total_unsheltered']
    tot_dist = r['overall_avg_distance']
    
    # Extract Men Stats
    m_stats = r['gender_breakdown']['men']
    m_shelt = m_stats['assigned']
    m_un = m_stats['unsheltered']
    m_dist = m_stats['avg_dist']
    
    # Extract Women Stats
    w_stats = r['gender_breakdown']['women']
    w_shelt = w_stats['assigned']
    w_un = w_stats['unsheltered']
    w_dist = w_stats['avg_dist']

    print(
        f"{r['month']:<6} | "
        f"{tot_un:<7.0f} {tot_dist:<7.3f} || "
        f"{m_shelt:<6.0f} {m_un:<8.0f} {m_dist:<7.3f} || "
        f"{w_shelt:<6.0f} {w_un:<8.0f} {w_dist:<7.3f}"
    )


                                           YEARLY SUMMARY                                           
Month  | TOTAL           || MEN                     || WOMEN                  
       | Unshelt AvgKM   || Shelt  Unshelt  AvgKM   || Shelt  Unshelt  AvgKM  
----------------------------------------------------------------------------------------------------
1      | 5535    9.722   || 1757   3716     11.062  || 758    1819     6.617  
2      | 5571    7.968   || 1757   3729     6.303   || 758    1842     11.827 
3      | 5065    7.220   || 1757   3388     6.957   || 758    1677     7.830  
4      | 5079    7.545   || 1757   3371     7.264   || 758    1708     8.197  
5      | 4680    9.331   || 1757   3066     9.922   || 758    1614     7.960  
6      | 4526    7.645   || 1757   2939     6.689   || 758    1587     9.861  
7      | 4455    7.633   || 1757   2852     7.406   || 758    1603     8.159  
8      | 4430    9.650   || 1757   2836     10.512  || 758    1594     7.653  
9      

#### Correct Sim Greedy Data

In [6]:
results = [
    {
        "month": 1,
        "total_unsheltered": 5535,
        "overall_avg_distance": 6.063,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3716, "avg_dist": 5.736},
            "women": {"assigned": 758, "unsheltered": 1819, "avg_dist": 6.819}
        }
    },
    {
        "month": 2,
        "total_unsheltered": 5571,
        "overall_avg_distance": 6.074,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3729, "avg_dist": 5.744},
            "women": {"assigned": 758, "unsheltered": 1842, "avg_dist": 6.839}
        }
    },
    {
        "month": 3,
        "total_unsheltered": 5065,
        "overall_avg_distance": 5.672,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3388, "avg_dist": 5.251},
            "women": {"assigned": 758, "unsheltered": 1677, "avg_dist": 6.649}
        }
    },
    {
        "month": 4,
        "total_unsheltered": 5079,
        "overall_avg_distance": 5.664,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3371, "avg_dist": 5.218},
            "women": {"assigned": 758, "unsheltered": 1708, "avg_dist": 6.699}
        }
    },
    {
        "month": 5,
        "total_unsheltered": 4680,
        "overall_avg_distance": 5.496,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3066, "avg_dist": 5.042},
            "women": {"assigned": 758, "unsheltered": 1614, "avg_dist": 6.546}
        }
    },
    {
        "month": 6,
        "total_unsheltered": 4526,
        "overall_avg_distance": 5.512,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2939, "avg_dist": 5.075},
            "women": {"assigned": 758, "unsheltered": 1587, "avg_dist": 6.524}
        }
    },
    {
        "month": 7,
        "total_unsheltered": 4455,
        "overall_avg_distance": 5.537,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2852, "avg_dist": 5.105},
            "women": {"assigned": 758, "unsheltered": 1603, "avg_dist": 6.539}
        }
    },
    {
        "month": 8,
        "total_unsheltered": 4430,
        "overall_avg_distance": 5.538,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2836, "avg_dist": 5.110},
            "women": {"assigned": 758, "unsheltered": 1594, "avg_dist": 6.531}
        }
    },
    {
        "month": 9,
        "total_unsheltered": 4260,
        "overall_avg_distance": 5.488,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2732, "avg_dist": 5.054},
            "women": {"assigned": 758, "unsheltered": 1528, "avg_dist": 6.494}
        }
    },
    {
        "month": 10,
        "total_unsheltered": 4251,
        "overall_avg_distance": 5.437,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2705, "avg_dist": 4.979},
            "women": {"assigned": 758, "unsheltered": 1546, "avg_dist": 6.499}
        }
    },
    {
        "month": 11,
        "total_unsheltered": 4540,
        "overall_avg_distance": 5.513,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 2934, "avg_dist": 5.071},
            "women": {"assigned": 758, "unsheltered": 1606, "avg_dist": 6.538}
        }
    },
    {
        "month": 12,
        "total_unsheltered": 5514,
        "overall_avg_distance": 6.052,
        "gender_breakdown": {
            "men": {"assigned": 1757, "unsheltered": 3690, "avg_dist": 5.721},
            "women": {"assigned": 758, "unsheltered": 1824, "avg_dist": 6.819}
        }
    }
]

# ==========================================
# 4. FINAL SUMMARY
# ==========================================
print("\n" + "="*100)
print(f"{'YEARLY SUMMARY':^100}")
print("="*100)

# Header Row
print(f"{'Month':<6} | {'TOTAL':<15} || {'MEN':<23} || {'WOMEN':<23}")
print(f"{'':<6} | {'Unshelt':<7} {'AvgKM':<7} || {'Shelt':<6} {'Unshelt':<8} {'AvgKM':<7} || {'Shelt':<6} {'Unshelt':<8} {'AvgKM':<7}")
print("-" * 100)

for r in results:
    tot_un = r['total_unsheltered']
    tot_dist = r['overall_avg_distance']
    
    m_stats = r['gender_breakdown']['men']
    m_shelt = m_stats['assigned']
    m_un = m_stats['unsheltered']
    m_dist = m_stats['avg_dist']
    
    w_stats = r['gender_breakdown']['women']
    w_shelt = w_stats['assigned']
    w_un = w_stats['unsheltered']
    w_dist = w_stats['avg_dist']

    print(
        f"{r['month']:<6} | "
        f"{tot_un:<7.0f} {tot_dist:<7.3f} || "
        f"{m_shelt:<6.0f} {m_un:<8.0f} {m_dist:<7.3f} || "
        f"{w_shelt:<6.0f} {w_un:<8.0f} {w_dist:<7.3f}"
    )


                                           YEARLY SUMMARY                                           
Month  | TOTAL           || MEN                     || WOMEN                  
       | Unshelt AvgKM   || Shelt  Unshelt  AvgKM   || Shelt  Unshelt  AvgKM  
----------------------------------------------------------------------------------------------------
1      | 5535    6.063   || 1757   3716     5.736   || 758    1819     6.819  
2      | 5571    6.074   || 1757   3729     5.744   || 758    1842     6.839  
3      | 5065    5.672   || 1757   3388     5.251   || 758    1677     6.649  
4      | 5079    5.664   || 1757   3371     5.218   || 758    1708     6.699  
5      | 4680    5.496   || 1757   3066     5.042   || 758    1614     6.546  
6      | 4526    5.512   || 1757   2939     5.075   || 758    1587     6.524  
7      | 4455    5.537   || 1757   2852     5.105   || 758    1603     6.539  
8      | 4430    5.538   || 1757   2836     5.110   || 758    1594     6.531  
9      